# Tutorial 4: Advanced features

##### In this tutorial, you will learn:
* _How to inspect Rigetti's devices using Forest API_
* _Extract and interpret the information about decoherence of the qubits_
* _Usage of the server-side compiler_
* _How to incorporate noise models to estimate performance of the circuit on the real chip_

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

In [ ]:
%matplotlib inline
# need newest pyQuil version due to fixed issues
!pip install --upgrade https://github.com/rigetticomputing/pyquil/archive/master.zip networkx matplotlib

In [ ]:
import os

API_KEY = 'YOUR API KEY GOES HERE'
USER_ID = 'YOUR USER ID GOES HERE'

PYQUIL_CONFIG = f"""
[Rigetti Forest]
url: https://api.rigetti.com/qvm
key: {API_KEY}
user_id: {USER_ID}
"""

with open(os.path.expanduser('~/.pyquil_config'), 'w') as f:
    f.write(PYQUIL_CONFIG)

### QPU status

The QPU devices can have different status and are not always immediatelly accessible. Luckily, you can inspect this information via Forest API.

#### Exercise 4.1
Use `pyquil.api` module to obtain list of available devices. Only list the devices that are currently online. What other states can devices be in?

### Chip topology

Knowing qubit topology of the chip you're working with is an important requirement to be able to design efficient circuits. Forest API provides an interface that exposes this information as well.


#### Exercise 4.2
Use <a href="https://pyquil.readthedocs.io/en/latest">Rigetti's documentation</a> to obtain and plot the topology of the `8Q-Agave` chip.

In [ ]:
import networkx
G = networkx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
networkx.draw(G, with_labels=True, font_weight='bold')

#### Exercise 4.3
Rigetti also has a newer chip, called `19Q-Acorn`, which is currently offline (as you learned in the exercise 4.1) due to high number of malfunctioning qubits/couplings after a power outage incident. Obtain and plot the topology of the `19Q-Acorn` chip. Note that malfunctioning parts of the chip are marked as "dead", make sure to exclude those.

In [ ]:
import networkx
G = networkx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
networkx.draw(G, with_labels=True, font_weight='bold')

### Decoherence and circuit depth

One of the main limiting factors if current near-term devices is actual decoherence of the qubits in the device. Decoherence can be loosely interpreted as loss of the quantum information in the qubit, and is caused by imperfections in the implementations of the quantum gates, or due to simple time-evolution of the system, as any current quantum chip is necessarily an open quantum system to a certain degree.

Generally, there are three sources of errors in the implementation of a given quantum circuit:
* actual qubit decoherence due to time-evolution (i.e. noise, interactions with environment)
* imprecise gate implementations (gates that do not have fidelities of 1)
* readout errors

#### $T_1$ and $T_2$ times
$T_1$ and $T_2$ relaxation times are one of the basic measures of performance of qubits in terms of decoherence. Roughly speaking:

* $T_1$ relaxation time (also called the "amplitude damping") is time during which the qubit decoheres from its excited state $\ket{1}$ to relaxed state $\ket{0}$.
* $T_2$ relaxation time (also called the "phase damping") can be measured using the following experiment: First you put the qubit into the superposition state $\frac{\ket{0}+\ket{1}}{\sqrt{2}}$ using a Hadamard gate, then you wait for $t_2$ and apply a Hadamard gate again. Without any noise, you would expect the state to be $\ket{0}$ again (remember $H^2 = I$), but if something caused the qubit to decohere to either $\ket{0}$ or $\ket{1}$ in the meantime, you will obtain the $\frac{\ket{0}\pm\ket{1}}{\sqrt{2}}$ instead.


#### Exercise 4.4

Use the device specifications of the $T_1$ times `8Q-Agave` chip obtained via the API, determine a simple upper bound on the single-qubit and two-qubit circuit depth on qubit 7 (assuming zero readout error and perfect-fidelity gates).

The average gate execution time of single qubit gate is 50ns and 150ns for the case of two-qubit gate.

#### Noise models
Simple estimation as given in the previous exercise provide a generous upper bound, since they neglect other source of errors in the computation, such as the readout errors or gate fidelities. To use a noise model, simply specify a device during definition of the QVMConnection object.

#### Exercise 4.5

Implement a simple circuit consisting of 20 alternating CNOT gates (swtiching between control qubit on 0 and 1). Empirically measure its performance (how many runs out of 1000 did not end up in the expected state).

Increase the number of gates to 100 and then 200. How do you interpret the results?

### Manually triggered compilation

In the usual setting of submitting jobs to the QPU, the compilation is happening transparently, without the user of the Forest API needing to know that the circuit they constructed is decomposed into sequence of gate operations that are actually implemented on the hardware.

Similiarly, in order to accurately simulate the device with the noise, the circuits also have to be compiled to the fundamental gate set implemented by the chip, since those gates are the ones we have experimental information about.

Given both these cases, it is useful to be able to actually inspect and analyze the compiled version of the circuit - since that is what is exactly being run on the device.

#### Exercise 4.6

Compile circuit from the previous exercise using the CompilerConnection object. Inspect the compiler circuit, and figure out if anything is surprising. Contrast with the result of the previous exercise. Does this explain any surprising results?

Additionally, figure out the following parameters of the compiled circuit:
* gate depth
* two-qubit gate depth
* total number of gates used

#### Exercise 4.7
Inspect the following circuit. Compile it to the `8Q-Agave` device. How does the compiled version differ from the uncompiled one?

```
H 2
H 4
H 7
CNOT 2 4
CNOT 4 7
CNOT 2 4
H 7
H 4
H 2
```